# 패키지불러오기

In [2]:
#이상치제거6파일에서 밑에서부터 620개를 선택
from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cmc
import numpy as np
import pandas as pd 
from colormath.color_objects import sRGBColor, XYZColor, LabColor
from colormath.color_conversions import convert_color
import pandas as pd
import math
import numpy as np
import numpy as np
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from PIL import Image
from matplotlib import pyplot as plt
import copy
from sklearn.preprocessing import LabelEncoder
import pandas.core.algorithms as algos
from pandas import Series
#from pandas import tools

ModuleNotFoundError: No module named 'colormath'

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.linear_model import LinearRegression
from tensorflow import keras
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
# gpu 상태 확인 및 메모리 사용만큼 할당
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
import webcolors
from colormath.color_objects import LabColor, sRGBColor 
from colormath.color_conversions import convert_color 
from colormath.color_diff import delta_e_cie2000
from colormath import color_diff_matrix
import matplotlib.colors as mcolors

from math import sqrt
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xlrd
import openpyxl
from pandasql import sqldf

# 필요함수

In [ ]:
def paste_python(aa,a):
    for i in range(len(aa)):
        aa[i]=a+aa[i]
    return(aa)

In [ ]:
def paste_python2(a,aa):
    for i in range(len(aa)):
        aa[i]=aa[i]+a
    return(aa)

In [ ]:
# lab를 rgb로 변환하는 함수
def lab_to_rgb(data):
    r_list=[]
    g_list=[]
    b_list=[]
    for i in range(len(data)):
        lab=LabColor(data[data.columns[0]][data.index[i]],data[data.columns[1]][data.index[i]],data[data.columns[2]][data.index[i]])#lab좌표
        rgb=convert_color(lab,sRGBColor)#rgb좌표
        rgb_list=[color for color in rgb.get_value_tuple()]
        r_list.append(rgb_list[0])
        g_list.append(rgb_list[1])
        b_list.append(rgb_list[2])
    df = pd.DataFrame(
                {'r' : r_list,
                 'g' : g_list,
                 'b' : b_list}
                )
    df.index=data.index
    return(df)

In [ ]:
def color_diff(pred_TRANS,Y_TRANS):
    diff = []
    for i in Y_TRANS.index:
        pred = pred_TRANS.loc[i]
        true = Y_TRANS.loc[i]
        p_lab = LabColor(pred[0],pred[1],pred[2])
        t_lab = LabColor(true[0],true[1],true[2])
        delta = delta_e_cmc(p_lab , t_lab)
        diff.append(delta)
    diff=pd.DataFrame(diff)
    diff.index=Y_TRANS.index
    return(diff)

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
    # 첨 class 선언신 변수생성
    def __init__(self,input_units, num_unit,output_unit,lams,**kwargs,
                ):
        super(AttentionLayer, self).__init__(**kwargs)
        self.input_units = input_units
        self.num_unit = num_unit
        self.output_unit = output_unit
        self.lambdas = lams
        
        
        
        
    def get_config(self):
        
        config = super().get_config().copy()
        config.update({
            'input_units': self.input_units,
            'num_unit': self.num_unit,
            'output_unit': self.output_unit,
            'lambdas': self.lambdas,
        })
        return config 
        
    def build(self, input_shape):  
        self.kernel1 = self.add_weight("kernel1",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape =[self.input_units,self.num_unit])
        self.bias1 = self.add_weight("bias1",initializer = tf.zeros_initializer(),shape=[1,self.num_unit])
        
        
        
        self.kernel2 = self.add_weight("kernel2",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape = [self.num_unit,self.output_unit])
        self.bias2 = self.add_weight("bias2",initializer = tf.zeros_initializer(),shape=[1,self.output_unit])
        
                    
        
    def call(self, inputs):
        shape = [ tf.shape(inputs)[k] for k in range(len(inputs.shape))]
        
        shape_list = [shape[0]] + [1 for i in range(len(inputs.shape)-1)]
        kernel1_ = tf.tile(tf.expand_dims(self.kernel1,0),shape_list)
        kernel2_ = tf.tile(tf.expand_dims(self.kernel2,0),shape_list)
        
        h1 = tf.matmul(inputs,kernel1_) + self.bias1
        h1 = tf.nn.tanh(h1)
        out = tf.matmul(h1,kernel2_) + self.bias2
        out = tf.nn.softmax(out,axis=1)
        flat_output_total = tf.multiply(inputs,tf.expand_dims(out[:,:,0],axis=2))
        flat_output_total = tf.reduce_sum(flat_output_total,axis=1)
        for i in range(1,self.output_unit):
            flat_output= tf.multiply(inputs,tf.expand_dims(out[:,:,i],axis=2))
            flat_output = tf.reduce_sum(flat_output,axis=1)
            flat_output_total = tf.concat([flat_output_total,flat_output],axis=1)
        
        return flat_output_total

In [ ]:
    # CIE94 Keras Custom Loss
    def cie94(y_true, y_pred):
        """Calculate color difference by using CIE94 formulae

        See http://en.wikipedia.org/wiki/Color_difference or
        http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

        cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
        >>> 58.0
        cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
        >>> 0.0
        """

        L1, a1, b1 = y_true[:,0],y_true[:,1],y_true[:,2]
        L2, a2, b2 = y_pred[:,0],y_pred[:,1],y_pred[:,2]


        C1 = K.sqrt(K.pow(a1,2) + K.pow(b1,2))
        C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
        delta_L = L1 - L2
        delta_C = C1 - C2
        delta_a = a1 - a2
        delta_b = b1 - b2
        delta_H_square = K.pow(delta_a,2) + K.pow(delta_b,2) - K.pow(delta_C,2)
        loss_a = (K.sqrt(K.pow(delta_L,2)
                + K.pow(delta_C,2) / K.pow(1.0 + 0.045 * C1,2)
                + delta_H_square / K.pow(1.0 + 0.015 * C1,2)))
        return loss_a

In [ ]:
#전처리_조제=dataframe
#전처리_조제명=dataframe_names
#전처리_조제량=dataframe_amount
def pivot_PET(dataframe):
    whe=[]
    for i in dataframe.columns:
        if '명' in i:
            whe.append(1)
        else:
            whe.append(0)
    whe=np.array(whe)
    
    dataframe_names=dataframe.columns[np.where(whe==1)]
    dataframe_amount=dataframe.columns[np.where(whe==0)]
    
    dataframe_names_df=dataframe[dataframe_names]
    dataframe_amount_df=dataframe[dataframe_amount]

    
    dataframe_names_unique=[]
    for i in dataframe_names:
        dataframe_names_unique.extend(dataframe[i].unique())

    col_names=pd.DataFrame(dataframe_names_unique)[0].unique()
    col_names=col_names[~pd.isnull(col_names)]
    dataframe_reshape=pd.DataFrame(index=dataframe.index,columns=col_names)
    
    for i in range(len(dataframe_reshape)):
        for j in range(len(dataframe_names_df.columns)):
            if pd.isna(dataframe_names_df.loc[i][j])==False:
                dataframe_reshape.loc[i][np.where(dataframe_reshape.columns==dataframe_names_df.loc[i][j])[0][0]]=dataframe_amount_df.loc[i][j]#[0][0]는고정
            else:
                continue
    return(dataframe_reshape)

# 디렉토리설정

In [ ]:
direc='D:/ipy/염색 공정 최적화 - 테이큰소프트 용역/데이터/'

# 데이터 불러오기

In [ ]:
data=pd.read_excel(direc+'기초 데이터.xlsx')#,encoding='euc-kr')
data=data[data.columns[1:]]
data=data.drop([1,],axis=0)
data=data.reset_index()
data=data[data.columns[1:]]
data

# 변수명변경

In [ ]:
#경사 전까지 컬렴명변경
list_col1=list(data.loc[0][np.where(data.columns=='Unnamed: 1')[0][0]:np.where(data.columns=='경사')[0][0]])
list_col1.extend(data.columns[np.where(data.columns=='경사')[0][0]:])
data.columns=list_col1
np.array(data.columns)

In [ ]:
#경사이후, 위사이전까지 컬럼명 변경
list_col1=list(data.loc[0][np.where(data.columns=='번호')[0][0]:np.where(data.columns=='경사')[0][0]])
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='경사')[0][0]:np.where(data.columns=='위사')[0][0]]),'경사_'))
list_col1.extend(data.columns[np.where(data.columns=='위사')[0][0]:])
data.columns=list_col1
np.array(data.columns)

In [ ]:
list(data.loc[0][np.where(data.columns=='번호')[0][0]:np.where(data.columns=='위사')[0][0]])

In [ ]:
#위사이후, 제직이전까지 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='위사')[0][0]])
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='위사')[0][0]:np.where(data.columns=='제직')[0][0]]),'위사_'))
list_col1.extend(data.columns[np.where(data.columns=='제직')[0][0]:])
data.columns=list_col1
np.array(data.columns)

In [ ]:
#제직이후, Pilot전처리 이전까지 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='제직')[0][0]])
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='제직')[0][0]:np.where(data.columns=='Pilot전처리')[0][0]]),'제직_'))
list_col1.extend(data.columns[np.where(data.columns=='Pilot전처리')[0][0]:])
data.columns=list_col1
np.array(data.columns)

In [ ]:
#Pilot전처리 이후, Pilot 후가공 공정 후 물성까지 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='Pilot전처리')[0][0]])
list_col1.extend(list(data.loc[0][np.where(data.columns=='Pilot전처리')[0][0]:np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]]))
list_col1.extend(data.columns[np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]:])
data.columns=list_col1
np.array(data.columns)

In [ ]:
#검사물성 컬럼명 변경
list_col1=list(data.columns[np.where(data.columns=='번호')[0][0]:np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]])

list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Pilot 후가공 공정 후 물성')[0][0]:np.where(data.columns=='Pilot 전처리 공정 후 물성')[0][0]]),'Pilot 후가공 공정 후 물성_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Pilot 전처리 공정 후 물성')[0][0]:np.where(data.columns=='Lab 후처리 환원세정 전 검사')[0][0]]),'Pilot 전처리 공정 후 물성_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Lab 후처리 환원세정 전 검사')[0][0]:np.where(data.columns=='Lab 후처리 환원세정 후 검사')[0][0]]),'Lab 후처리 환원세정 전 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Lab 후처리 환원세정 후 검사')[0][0]:np.where(data.columns=='잔욕염색 검사')[0][0]]),'Lab 후처리 환원세정 후 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='잔욕염색 검사')[0][0]:np.where(data.columns=='Lab 후가공 후 검사')[0][0]]),'잔욕염색 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='Lab 후가공 후 검사')[0][0]:np.where(data.columns=='내광성 시험 전 검사')[0][0]]),'Lab 후가공 후 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='내광성 시험 전 검사')[0][0]:np.where(data.columns=='내광성 시험 후 검사')[0][0]]),'내광성 시험 전 검사_'))
list_col1.extend(paste_python(list(data.loc[0][np.where(data.columns=='내광성 시험 후 검사')[0][0]:(np.where(data.columns=='내광성 시험 후 검사')[0][0]+6)]),'내광성 시험 후 검사_'))

data.columns=list_col1
np.array(data.columns)

In [ ]:
list_col1

In [ ]:
data=data.drop([0],axis=0)
data=data.reset_index()
data=data[data.columns[1:]]
data

In [ ]:
#data.to_csv(direc+'원본데이터.csv',encoding='euc-kr')

# 데이터전처리

In [ ]:
#377번 데이터 제거
#data=data.loc[0:375]
#data

In [ ]:
np.array(data.columns)

In [71]:
#필요없는 변수 제거
data=data.drop(['번호','시료번호','내광성시험분석','접수번호No.','업체명','염색법','기계분류ID','기계분류','원료ID','제직_수량','Pilot 전처리 기계장비ID',
                 'Pilot 전처리 건조장비ID','Lab 염색 기계 ID','Lab 후처리 기계ID','Lab 후가공 패더장비 ID','Lab 후가공 텐터장비ID',
                  '내광성 시험 전 검사_L*', '내광성 시험 전 검사_a*', '내광성 시험 전 검사_b*',
                   '내광성 시험 후 검사_L*', '내광성 시험 후 검사_a*', '내광성 시험 후 검사_b*','내광성 시험 후 검사_색차',
                'Lab 후가공 후 검사_중량', 'Lab 후가공 후 검사_경사_밀도', 'Lab 후가공 후 검사_위사_밀도',
                 'Lab 후가공 후 검사_K/S', '내광성 시험 후 검사_내광성등급', '내광성 시험 후 검사_비고',
               'Pilot 전처리 공정 후 물성_중량', 'Pilot 전처리 공정 후 물성_경사_밀도',
               'Pilot 전처리 공정 후 물성_위사_밀도', 'Pilot 전처리 공정 후 물성_원단 폭',
#Pilot 변수들은 아직 사용하지 않음.             
       'Lab 후가공 pick-up율', 'Pilot 배합 원단 중량',
       'Pilot 배합 염료 #1 명', 'Pilot 배합 염료 #1 제조사명', 'Pilot 배합 염료 #1 종류',
       'Pilot 배합 염료 #1 타입', 'Pilot 배합 염료 #1 투입량', 'Pilot 배합 염료 #2 명',
       'Pilot 배합 염료 #2 제조사명', 'Pilot 배합 염료 #2 종류', 'Pilot 배합 염료 #2 타입',
       'Pilot 배합 염료 #2 투입량', 'Pilot 배합 염료 #3 명', 'Pilot 배합 염료 #3 제조사명',
       'Pilot 배합 염료 #3 종류', 'Pilot 배합 염료 #3 타입', 'Pilot 배합 염료 #3 투입량',
       'Pilot 배합 염료 #4 명', 'Pilot 배합 염료 #4 제조사명', 'Pilot 배합 염료 #4 종류',
       'Pilot 배합 염료 #4 타입', 'Pilot 배합 염료 #4 투입량', 'Pilot 배합 액비',
       'Pilot 배합 용수량', 'Pilot 배합 조제 #1 명', 'Pilot 배합 조제 #1 제조사명',
       'Pilot 배합 조제 #1 종류', 'Pilot 배합 조제 #1 투입량', 'Pilot 배합 조제 #2 명',
       'Pilot 배합 조제 #2 제조사명', 'Pilot 배합 조제 #2 종류', 'Pilot 배합 조제 #2 투입량',
       'Pilot 배합 조제 #3 명', 'Pilot 배합 조제 #3 제조사명', 'Pilot 배합 조제 #3 종류',
       'Pilot 배합 조제 #3 투입량', 'Pilot 배합 조제 #4 명', 'Pilot 배합 조제 #4 제조사명',
       'Pilot 배합 조제 #4 종류', 'Pilot 배합 조제 #4 투입량', 'Pilot 염색 장비ID',
       'Pilot 염색 장비명', 'Pilot 염색 시작온도', 'Pilot 염색 상승속도 #1',
       'Pilot 염색 상승온도 #1', 'Pilot 염색 상승온도 #1 유지시간', 'Pilot 염색 상승속도 #2',
       'Pilot 염색 상승온도 #2', 'Pilot 염색 상승온도 #2 유지시간', 'Pilot 염색 상승속도 #3',
       'Pilot 염색 상승온도 #3', 'Pilot 염색 상승온도 #3 유지시간', 'Pilot 염색 하강속도 #1',
       'Pilot 염색 하강온도 #1', 'Pilot 염색 하강온도 #1 유지시간', 'Pilot 염색 하강속도 #2',
       'Pilot 염색 하강온도 #2', 'Pilot 염색 하강온도 #2 유지시간', 'Pilot 염색 하강속도 #3',
       'Pilot 염색 하강온도 #3', 'Pilot 염색 하강온도 #3 유지시간', 'Pilot 염색 종료속도',
       'Pilot 염색 종료온도', 'Pilot 염색 종료온도 유지시간', 'Pilot 염색 펌프압력',
       'Pilot 염색 릴속도', 'Pilot 염색 재염횟수', 'Pilot 후처리 장비ID', 'Pilot 후처리 장비명',
       'Pilot 후처리 조제 #1 명', 'Pilot 후처리 조제 #1 제조사명', 'Pilot 후처리 조제 #1 종류',
       'Pilot 후처리 조제 #1 투입량', 'Pilot 후처리 조제 #2 명', 'Pilot 후처리 조제 #2 제조사명',
       'Pilot 후처리 조제 #2 종류', 'Pilot 후처리 조제 #2 투입량', 'Pilot 후처리 환원세정 액비',
       'Pilot 후처리 환원세정 용수량', 'Pilot 후처리 환원세정 시작온도',
       'Pilot 후처리 환원세정 상승속도 #1', 'Pilot 후처리 환원세정 상승온도 #1',
       'Pilot 후처리 환원세정 상승온도 #1 유지시간', 'Pilot 후처리 환원세정 상승속도 #2',
       'Pilot 후처리 환원세정 상승온도 #2', 'Pilot 후처리 환원세정 상승온도 #2 유지시간',
       'Pilot 후처리 환원세정 상승속도 #3', 'Pilot 후처리 환원세정 상승온도 #3',
       'Pilot 후처리 환원세정 상승온도 #3 유지시간', 'Pilot 후처리 환원세정 하강속도 #1',
       'Pilot 후처리 환원세정 하강온도 #1', 'Pilot 후처리 환원세정 하강온도 #1 유지시간',
       'Pilot 후처리 환원세정 하강속도 #2', 'Pilot 후처리 환원세정 하강온도 #2',
       'Pilot 후처리 환원세정 하강온도 #2 유지시간', 'Pilot 후처리 환원세정 하강속도 #3',
       'Pilot 후처리 환원세정 하강온도 #3', 'Pilot 후처리 환원세정 하강온도 #3 유지시간',
       'Pilot 후처리 환원세정 종료속도', 'Pilot 후처리 환원세정 종료온도',
       'Pilot 후처리 환원세정 종료온도 유지시간', 'Pilot 후처리 수세온도 #1',
       'Pilot 후처리 수세시간 #1', 'Pilot 후처리 수세횟수 #1', 'Pilot 후처리 수세온도 #2',
       'Pilot 후처리 수세시간 #2', 'Pilot 후처리 수세횟수 #2', 'Pilot 후처리 수세온도 #3',
       'Pilot 후처리 수세시간 #3', 'Pilot 후처리 수세횟수 #3', 'Pilot 후가공 장비ID',
       'Pilot 후가공 장비명', 'Pilot 후가공 원단 투입량', 'Pilot 후가공 조제 명',
       'Pilot 후가공 조제 제조사명', 'Pilot 후가공 조제 종류', 'Pilot 후가공 조제 투입량',
       'Pilot 후가공 오버피드', 'Pilot 후가공 pick-up율', 'Pilot 후가공 오른쪽 패더압력',
       'Pilot 후가공 왼쪽 패더압력', 'Pilot 후가공 오른쪽 실제포온도 #1',
       'Pilot 후가공 왼쪽 실제포온도 #1', 'Pilot 후가공 오른쪽 세팅챔버온도 #1',
       'Pilot 후가공 왼쪽 세팅챔버온도 #1', 'Pilot 후가공 건조속도 #1', 'Pilot 후가공 건조열풍 #1',
       'Pilot 후가공 오른쪽 실제포온도 #2', 'Pilot 후가공 왼쪽 실제포온도 #2',
       'Pilot 후가공 오른쪽 세팅챔버온도 #2', 'Pilot 후가공 왼쪽 세팅챔버온도 #2',
       'Pilot 후가공 건조속도 #2', 'Pilot 후가공 건조열풍 #2', 'Pilot 후가공 오른쪽 실제포온도 #3',
       'Pilot 후가공 왼쪽 실제포온도 #3', 'Pilot 후가공 오른쪽 세팅챔버온도 #3',
       'Pilot 후가공 왼쪽 세팅챔버온도 #3', 'Pilot 후가공 건조속도 #3', 'Pilot 후가공 건조열풍 #3',
       'Pilot 후가공 오른쪽 실제포온도 #4', 'Pilot 후가공 왼쪽 실제포온도 #4',
       'Pilot 후가공 오른쪽 세팅챔버온도 #4', 'Pilot 후가공 왼쪽 세팅챔버온도 #4',
       'Pilot 후가공 건조속도 #4', 'Pilot 후가공 건조열풍 #4', 'Pilot 후가공 오른쪽 실제포온도 #5',
       'Pilot 후가공 왼쪽 실제포온도 #5', 'Pilot 후가공 오른쪽 세팅챔버온도 #5',
       'Pilot 후가공 왼쪽 세팅챔버온도 #5', 'Pilot 후가공 건조속도 #5', 'Pilot 후가공 건조열풍 #5',
       'Pilot 후가공 오른쪽 실제포온도 #6', 'Pilot 후가공 왼쪽 실제포온도 #6',
       'Pilot 후가공 오른쪽 세팅챔버온도 #6', 'Pilot 후가공 왼쪽 세팅챔버온도 #6',
       'Pilot 후가공 건조속도 #6', 'Pilot 후가공 건조열풍 #6', 'Pilot 후가공 오른쪽 실제포온도 #7',
       'Pilot 후가공 왼쪽 실제포온도 #7', 'Pilot 후가공 오른쪽 세팅챔버온도 #7',
       'Pilot 후가공 왼쪽 세팅챔버온도 #7', 'Pilot 후가공 건조속도 #7', 'Pilot 후가공 건조열풍 #7',
       'Pilot 후가공 공정 후 물성_중량', 'Pilot 후가공 공정 후 물성_원단 폭',
#검사쪽은 공정 이후에 발생가능하므로 lab후가공 후 lab 및 k/s를 제외하고 ai모델개발 대상에서 제외
       'Lab 후처리 환원세정 전 검사_L*', 'Lab 후처리 환원세정 전 검사_a*',
       'Lab 후처리 환원세정 전 검사_b*', 'Lab 후처리 환원세정 전 검사_K/S',
       'Lab 후처리 환원세정 후 검사_L*', 'Lab 후처리 환원세정 후 검사_a*',
       'Lab 후처리 환원세정 후 검사_b*', 'Lab 후처리 환원세정 후 검사_K/S'
               ],axis=1)
data

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_성상,경사_형상,위사_소재명,위사_섬도,...,잔욕염색 검사_b*,잔욕염색 검사_K/S,Lab 후가공 후 검사_L*,Lab 후가공 후 검사_a*,Lab 후가공 후 검사_b*,Lab 후가공 후 검사_내마모성,Lab 후가공 후 검사_세탁견뢰도_시료,Lab 후가공 후 검사_세탁견뢰도_백면포,Lab 후가공 후 검사_습식마찰착색성,Lab 후가공 후 검사_건식마찰착색성
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,2.55,0.0415,76.05,2.82,-7.77,4,4.5,4,4.5,4.5
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,2.5,0.0396,75.88,2.92,-8.14,4,4.5,4,4.5,4.5
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,2.87,0.0412,76.05,12.78,-5.2,4,4.5,4,4.5,4.5
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,2.85,0.0435,75.94,12.59,-4.58,4,4.5,4,4.5,4.5
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,2.53,0.0398,78.3,-9.18,2.82,4,4.5,4,4.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,1.5226,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3164,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-3.6497,0.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-4.2008,0.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,150*2 INT,Filaments,PET 재생 섬유,300,...,-7.7309,0.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
np.array(data.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_성상',
       '경사_형상', '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도',
       '위사_성상', '위사_형상', '제직_원단구조', '제직_조직', '제직_중량_생지', '제직_폭_생지',
       'Pilot 전처리 기계장비명', 'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명',
       'Pilot 전처리 조제 #1제조사명', 'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량',
       'Pilot 전처리 조제 #2 제품명', 'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류',
       'Pilot 전처리 조제 #2투입량', 'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명',
       'Pilot 전처리 조제 #3 종류', 'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도',
       'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간',
       'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간',
       'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간',
       'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간',
       'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간',
       'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3'

# 1. 전처리

In [73]:
전처리=data[[#'Pilot 전처리 기계장비명',
       'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명',
       'Pilot 전처리 조제 #1제조사명', 'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량',
       'Pilot 전처리 조제 #2 제품명', 'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류',
       'Pilot 전처리 조제 #2투입량', 'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명',
       'Pilot 전처리 조제 #3 종류', 'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도',
       'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간',
       'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간',
       'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간',
       'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간',
       'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간',
       'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 #3 유지시간',
       'Pilot 전처리 종료속도', 'Pilot 전처리 종료온도', 'Pilot 전처리 종료온도 유지시간',
       'Pilot 전처리 수세온도', 'Pilot 전처리 수세시간', 'Pilot 전처리 수세횟수',
       'Pilot 전처리 건조장비', 'Pilot 전처리 오버피드', 'Pilot 전처리 오른쪽 패더압력',
       'Pilot 전처리 왼쪽 패더압력', 'Pilot 전처리 오른쪽 실제포온도 #1',
       'Pilot 전처리 왼쪽 실제포온도 #1', 'Pilot 전처리 오른쪽 세팅챔버온도 #1',
       'Pilot 전처리 왼쪽 세팅챔버온도 #1', 'Pilot 전처리 건조속도 #1', 'Pilot 전처리 건조열풍 #1',
       'Pilot 전처리 오른쪽 실제포온도 #2', 'Pilot 전처리 왼쪽 실제포온도 #2',
       'Pilot 전처리 오른쪽 세팅챔버온도 #2', 'Pilot 전처리 왼쪽 세팅챔버온도 #2',
       'Pilot 전처리 건조속도 #2', 'Pilot 전처리 건조열풍 #2', 'Pilot 전처리 오른쪽 실제포온도 #3',
       'Pilot 전처리 왼쪽 실제포온도 #3', 'Pilot 전처리 오른쪽 세팅챔버온도 #3',
       'Pilot 전처리 왼쪽 세팅챔버온도 #3', 'Pilot 전처리 건조속도 #3', 'Pilot 전처리 건조열풍 #3',
       'Pilot 전처리 오른쪽 실제포온도 #4', 'Pilot 전처리 왼쪽 실제포온도 #4',
       'Pilot 전처리 오른쪽 세팅챔버온도 #4', 'Pilot 전처리 왼쪽 세팅챔버온도 #4',
       'Pilot 전처리 건조속도 #4', 'Pilot 전처리 건조열풍 #4', 'Pilot 전처리 오른쪽 실제포온도 #5',
       'Pilot 전처리 왼쪽 실제포온도 #5', 'Pilot 전처리 오른쪽 세팅챔버온도 #5',
       'Pilot 전처리 왼쪽 세팅챔버온도 #5', 'Pilot 전처리 건조속도 #5', 'Pilot 전처리 건조열풍 #5',
       'Pilot 전처리 오른쪽 실제포온도 #6', 'Pilot 전처리 왼쪽 실제포온도 #6',
       'Pilot 전처리 오른쪽 세팅챔버온도 #6', 'Pilot 전처리 왼쪽 세팅챔버온도 #6',
       'Pilot 전처리 건조속도 #6', 'Pilot 전처리 건조열풍 #6', 'Pilot 전처리 오른쪽 실제포온도 #7',
       'Pilot 전처리 왼쪽 실제포온도 #7', 'Pilot 전처리 오른쪽 세팅챔버온도 #7',
       'Pilot 전처리 왼쪽 세팅챔버온도 #7', 'Pilot 전처리 건조속도 #7', 'Pilot 전처리 건조열풍 #7']]

In [74]:
len(전처리.columns)

84

In [75]:
전처리_조제=전처리[['Pilot 전처리 조제 #1 제품명','Pilot 전처리 조제 #1투입량',
            'Pilot 전처리 조제 #2 제품명','Pilot 전처리 조제 #2투입량',
           'Pilot 전처리 조제 #3 제품명','Pilot 전처리 조제 #3투입량']]


In [76]:
전처리_조제

,Pilot 전처리 조제 #1 제품명,Pilot 전처리 조제 #1투입량,Pilot 전처리 조제 #2 제품명,Pilot 전처리 조제 #2투입량,Pilot 전처리 조제 #3 제품명,Pilot 전처리 조제 #3투입량
0,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
1,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
2,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
4,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
...,...,...,...,...,...,...
3163,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3164,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3165,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1
3166,Sera Wash C-NEP,3,NaOH,5,CH3COOH,1


In [77]:
전처리_조제_df=pivot_PET(전처리_조제)
전처리_조제_df.columns='전처리_'+전처리_조제_df.columns
전처리_조제_df


,전처리_Sera Wash C-NEP,전처리_NaOH,전처리_CH3COOH
0,3,5,1
1,3,5,1
2,3,5,1
3,3,5,1
4,3,5,1
...,...,...,...
3163,3,5,1
3164,3,5,1
3165,3,5,1
3166,3,5,1


In [78]:
전처리_온도_df=data[['Pilot 전처리 시작온도',
       'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간',
       'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간',
       'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간',
       'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간',
       'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간',
       'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 #3 유지시간',
       'Pilot 전처리 종료속도', 'Pilot 전처리 종료온도', 'Pilot 전처리 종료온도 유지시간',
       'Pilot 전처리 수세온도', 'Pilot 전처리 수세시간', 'Pilot 전처리 수세횟수'
               ]]
전처리_온도_df

,Pilot 전처리 시작온도,Pilot 전처리 상승속도 #1,Pilot 전처리 상승온도 #1,Pilot 전처리 상승온도 #1 유지시간,Pilot 전처리 상승속도 #2,Pilot 전처리 상승온도 #2,Pilot 전처리 상승온도 #2 유지시간,Pilot 전처리 상승속도 #3,Pilot 전처리 상승온도 #3,Pilot 전처리 상승온도 #3 유지시간,...,Pilot 전처리 하강온도 #2 유지시간,Pilot 전처리 하강속도 #3,Pilot 전처리 하강온도 #3,Pilot 전처리 하강온도 #3 유지시간,Pilot 전처리 종료속도,Pilot 전처리 종료온도,Pilot 전처리 종료온도 유지시간,Pilot 전처리 수세온도,Pilot 전처리 수세시간,Pilot 전처리 수세횟수
0,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
1,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
2,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
3,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
4,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
3164,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
3165,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2
3166,45,5.5,95,60,0,95,0,0,95,0,...,0,0,95,0,0,95,0,80,10,2


In [79]:
#세팅챔버온도 및 건조속도, 건조풍속 평균냄


#실제포온도 삭제
전처리_건조_df=data[[
       'Pilot 전처리 오버피드', 
       'Pilot 전처리 오른쪽 패더압력','Pilot 전처리 왼쪽 패더압력', 
       'Pilot 전처리 오른쪽 세팅챔버온도 #1', 'Pilot 전처리 왼쪽 세팅챔버온도 #1', 'Pilot 전처리 건조속도 #1', 'Pilot 전처리 건조열풍 #1',
       'Pilot 전처리 오른쪽 세팅챔버온도 #2', 'Pilot 전처리 왼쪽 세팅챔버온도 #2', 'Pilot 전처리 건조속도 #2', 'Pilot 전처리 건조열풍 #2',
       'Pilot 전처리 오른쪽 세팅챔버온도 #3', 'Pilot 전처리 왼쪽 세팅챔버온도 #3', 'Pilot 전처리 건조속도 #3', 'Pilot 전처리 건조열풍 #3',
       'Pilot 전처리 오른쪽 세팅챔버온도 #4', 'Pilot 전처리 왼쪽 세팅챔버온도 #4', 'Pilot 전처리 건조속도 #4', 'Pilot 전처리 건조열풍 #4', 
       'Pilot 전처리 오른쪽 세팅챔버온도 #5', 'Pilot 전처리 왼쪽 세팅챔버온도 #5', 'Pilot 전처리 건조속도 #5', 'Pilot 전처리 건조열풍 #5',
       'Pilot 전처리 오른쪽 세팅챔버온도 #6', 'Pilot 전처리 왼쪽 세팅챔버온도 #6', 'Pilot 전처리 건조속도 #6', 'Pilot 전처리 건조열풍 #6',
       'Pilot 전처리 오른쪽 세팅챔버온도 #7', 'Pilot 전처리 왼쪽 세팅챔버온도 #7', 'Pilot 전처리 건조속도 #7', 'Pilot 전처리 건조열풍 #7'
]]

#세팅챔버온도 및 건조속도, 건조풍속 평균냄
전처리_건조_df['Pilot 전처리 세팅챔버온도']=전처리_건조_df.apply(lambda x : np.mean(x[['Pilot 전처리 오른쪽 세팅챔버온도 #1',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #2',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #3',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #4',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #5',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #6',
                                                                                        'Pilot 전처리 오른쪽 세팅챔버온도 #7',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #1',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #2',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #3',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #4',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #5',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #6',
                                                                                        'Pilot 전처리 왼쪽 세팅챔버온도 #7'
                                                                   ]]),axis=1)

전처리_건조_df['Pilot 전처리 건조속도']=전처리_건조_df.apply(lambda x : np.mean(x[[    'Pilot 전처리 건조속도 #1',
                                                                                        'Pilot 전처리 건조속도 #2',
                                                                                        'Pilot 전처리 건조속도 #3',
                                                                                        'Pilot 전처리 건조속도 #4',
                                                                                        'Pilot 전처리 건조속도 #5',
                                                                                        'Pilot 전처리 건조속도 #6',
                                                                                        'Pilot 전처리 건조속도 #7'
                                                                   ]]),axis=1)

전처리_건조_df['Pilot 전처리 건조열풍']=전처리_건조_df.apply(lambda x : np.mean(x[[    'Pilot 전처리 건조열풍 #1',
                                                                                        'Pilot 전처리 건조열풍 #2',
                                                                                        'Pilot 전처리 건조열풍 #3',
                                                                                        'Pilot 전처리 건조열풍 #4',
                                                                                        'Pilot 전처리 건조열풍 #5',
                                                                                        'Pilot 전처리 건조열풍 #6',
                                                                                        'Pilot 전처리 건조열풍 #7'
                                                                   ]]),axis=1)


전처리_건조_df=전처리_건조_df[[
       'Pilot 전처리 오버피드', 
       'Pilot 전처리 오른쪽 패더압력','Pilot 전처리 왼쪽 패더압력', 'Pilot 전처리 세팅챔버온도', 'Pilot 전처리 건조속도', 'Pilot 전처리 건조열풍'
                                ]]
#전처리_검사_df=data[['Pilot 전처리 공정 후 물성_중량',
#       'Pilot 전처리 공정 후 물성_원단 폭','Pilot 전처리 공정 후 물성_경사_밀도',
#       'Pilot 전처리 공정 후 물성_위사_밀도']]

전처리_건조_df

C:\Users\HP\AppData\Local\Temp/ipykernel_13468/1241419137.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  전처리_건조_df['Pilot 전처리 세팅챔버온도']=전처리_건조_df.apply(lambda x : np.mean(x[['Pilot 전처리 오른쪽 세팅챔버온도 #1',
C:\Users\HP\AppData\Local\Temp/ipykernel_13468/1241419137.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  전처리_건조_df['Pilot 전처리 건조속도']=전처리_건조_df.apply(lambda x : np.mean(x[[    'Pilot 전처리 건조속도 #1',
C:\Users\HP\AppData\Local\Temp/ipykernel_13468/1241419137.py:43: SettingWithCopyWarning: 
A value is try

,Pilot 전처리 오버피드,Pilot 전처리 오른쪽 패더압력,Pilot 전처리 왼쪽 패더압력,Pilot 전처리 세팅챔버온도,Pilot 전처리 건조속도,Pilot 전처리 건조열풍
0,1,3,3,170.0,20.0,1200.0
1,1,3,3,170.0,20.0,1200.0
2,1,3,3,170.0,20.0,1200.0
3,1,3,3,170.0,20.0,1200.0
4,1,3,3,170.0,20.0,1200.0
...,...,...,...,...,...,...
3163,1,3,3,170.0,20.0,1200.0
3164,1,3,3,170.0,20.0,1200.0
3165,1,3,3,170.0,20.0,1200.0
3166,1,3,3,170.0,20.0,1200.0


In [80]:
전처리=pd.concat([전처리_조제_df,전처리_온도_df,전처리_건조_df,
               #전처리_검사_df
              ],axis=1)
전처리

,전처리_Sera Wash C-NEP,전처리_NaOH,전처리_CH3COOH,Pilot 전처리 시작온도,Pilot 전처리 상승속도 #1,Pilot 전처리 상승온도 #1,Pilot 전처리 상승온도 #1 유지시간,Pilot 전처리 상승속도 #2,Pilot 전처리 상승온도 #2,Pilot 전처리 상승온도 #2 유지시간,...,Pilot 전처리 종료온도 유지시간,Pilot 전처리 수세온도,Pilot 전처리 수세시간,Pilot 전처리 수세횟수,Pilot 전처리 오버피드,Pilot 전처리 오른쪽 패더압력,Pilot 전처리 왼쪽 패더압력,Pilot 전처리 세팅챔버온도,Pilot 전처리 건조속도,Pilot 전처리 건조열풍
0,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
1,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
2,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
4,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3164,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3165,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0
3166,3,5,1,45,5.5,95,60,0,95,0,...,0,80,10,2,1,3,3,170.0,20.0,1200.0


In [81]:
np.array(data.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_성상',
       '경사_형상', '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도',
       '위사_성상', '위사_형상', '제직_원단구조', '제직_조직', '제직_중량_생지', '제직_폭_생지',
       'Pilot 전처리 기계장비명', 'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명',
       'Pilot 전처리 조제 #1제조사명', 'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량',
       'Pilot 전처리 조제 #2 제품명', 'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류',
       'Pilot 전처리 조제 #2투입량', 'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명',
       'Pilot 전처리 조제 #3 종류', 'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도',
       'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간',
       'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간',
       'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간',
       'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간',
       'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간',
       'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3'

# 2. 원단정보

In [82]:
data['경사_성상']

0       150*2 INT
1       150*2 INT
2       150*2 INT
3       150*2 INT
4       150*2 INT
          ...    
3163    150*2 INT
3164    150*2 INT
3165    150*2 INT
3166    150*2 INT
3167    150*2 INT
Name: 경사_성상, Length: 3168, dtype: object

In [83]:
data['경사_성상_섬도 #1']=data['경사_성상'].str[0:3]
data['경사_성상_섬도 #2']=data['경사_성상'].str[0:3]
data['경사_성상_섬도 #3']=0
data['경사_성상_섬도 #4']=0

data['경사_성상_비율 #1']=50
data['경사_성상_비율 #2']=50
data['경사_성상_비율 #3']=0
data['경사_성상_비율 #4']=0

data['경사_성상_특성']='Interlace'

data['위사_성상_섬도 #1']=data['위사_성상'].str[0:3]
data['위사_성상_섬도 #2']=data['위사_성상'].str[0:3]
data['위사_성상_섬도 #3']=0
data['위사_성상_섬도 #4']=0

data['위사_성상_비율 #1']=50
data['위사_성상_비율 #2']=50
data['위사_성상_비율 #3']=0
data['위사_성상_비율 #4']=0

data['위사_성상_특성']='Interlace'

data=data.drop(['경사_성상','위사_성상'],axis=1)
data

소재=data[['경사_소재명', 
         '경사_섬도', 
         '경사_필라멘트수', 
         '경사_광택', 
         '경사_특성', 
         '경사_밀도',
         '경사_형상', 
         '경사_성상_섬도 #1',
         '경사_성상_섬도 #2','경사_성상_섬도 #3','경사_성상_섬도 #4',
        '경사_성상_비율 #1',
         '경사_성상_비율 #2','경사_성상_비율 #3','경사_성상_비율 #4',
        '경사_성상_특성',
         '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도',
       '위사_형상', '위사_성상_섬도 #1','위사_성상_섬도 #2','위사_성상_섬도 #3','위사_성상_섬도 #4',
        '위사_성상_비율 #1','위사_성상_비율 #2','위사_성상_비율 #3','위사_성상_비율 #4',
         '위사_성상_특성',
        '제직_원단구조', '제직_조직','제직_중량_생지','제직_폭_생지']]
#소재=소재.fillna('none')
소재

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,위사_성상_섬도 #4,위사_성상_비율 #1,위사_성상_비율 #2,위사_성상_비율 #3,위사_성상_비율 #4,위사_성상_특성,제직_원단구조,제직_조직,제직_중량_생지,제직_폭_생지
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,237,1.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,242,1.37
3164,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,242,1.37
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,242,1.37
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,50,50,0,0,Interlace,WOVEN,TWILL,242,1.37


In [84]:
np.array(소재.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_형상',
       '경사_성상_섬도 #1', '경사_성상_섬도 #2', '경사_성상_섬도 #3', '경사_성상_섬도 #4',
       '경사_성상_비율 #1', '경사_성상_비율 #2', '경사_성상_비율 #3', '경사_성상_비율 #4',
       '경사_성상_특성', '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성',
       '위사_밀도', '위사_형상', '위사_성상_섬도 #1', '위사_성상_섬도 #2', '위사_성상_섬도 #3',
       '위사_성상_섬도 #4', '위사_성상_비율 #1', '위사_성상_비율 #2', '위사_성상_비율 #3',
       '위사_성상_비율 #4', '위사_성상_특성', '제직_원단구조', '제직_조직', '제직_중량_생지',
       '제직_폭_생지'], dtype=object)

# 3. 배합정보

In [85]:
배합=data[[ 'Lab 배합 염료 #1 명','Lab 배합 염료 #1 투입량',
            'Lab 배합 염료 #2 명','Lab 배합 염료 #2 투입량',
            'Lab 배합 염료 #3 명','Lab 배합 염료 #3 투입량',
            'Lab 배합 염료 #4 명','Lab 배합 염료 #4 투입량',
                     ]]
배합_df=pivot_PET(배합)
배합_df

#Dorosperse Brown K-3LR, Synofast 28가 없어서 추가해줘야함
#배합_df['Dorosperse Brown K-3LR']=np.nan
#배합_df['Synofast 28']=np.nan

,Dorosperse Red KKL,Dorosperse Yellow KKL,Dorosperse Blue KKL,Dianix Red AM-SLR,Dianix Yellow AM-2R,Dianix Blue AM-2G,Synolon Red AK,Synolon Yellow AK,Synolon Blue AK,Dorosperse B/K KKL,Dorosperse Dark Grey KKL,Dorosperse Brown K-3LR,Dianix Black AM-SLR,Dianix Grey AM-SLR,Dianix Yellow Brown AM-R,Synolon Black AK-NB,Synolon Dark Grey AK,Synofast 28
0,0.017,NaN,0.033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.017,NaN,0.033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.033,NaN,0.017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.033,NaN,0.017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.017,0.033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
배합_df.columns

Index(['Dorosperse Red KKL', 'Dorosperse Yellow KKL', 'Dorosperse Blue KKL',
       'Dianix Red AM-SLR', 'Dianix Yellow AM-2R', 'Dianix Blue AM-2G',
       'Synolon Red AK', 'Synolon Yellow AK', 'Synolon Blue AK',
       'Dorosperse B/K KKL', 'Dorosperse Dark Grey KKL',
       'Dorosperse Brown K-3LR', 'Dianix Black AM-SLR', 'Dianix Grey AM-SLR',
       'Dianix Yellow Brown AM-R', 'Synolon Black AK-NB',
       'Synolon Dark Grey AK', 'Synofast 28'],
      dtype='object')

In [87]:
배합_df=배합_df.fillna(0)
배합_df

,Dorosperse Red KKL,Dorosperse Yellow KKL,Dorosperse Blue KKL,Dianix Red AM-SLR,Dianix Yellow AM-2R,Dianix Blue AM-2G,Synolon Red AK,Synolon Yellow AK,Synolon Blue AK,Dorosperse B/K KKL,Dorosperse Dark Grey KKL,Dorosperse Brown K-3LR,Dianix Black AM-SLR,Dianix Grey AM-SLR,Dianix Yellow Brown AM-R,Synolon Black AK-NB,Synolon Dark Grey AK,Synofast 28
0,0.017,0.000,0.033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.017,0.000,0.033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.033,0.000,0.017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.033,0.000,0.017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.017,0.033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3164,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3165,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3166,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
배합_df.describe()

,Dorosperse Red KKL,Dorosperse Yellow KKL,Dorosperse Blue KKL,Dianix Red AM-SLR,Dianix Yellow AM-2R,Dianix Blue AM-2G,Synolon Red AK,Synolon Yellow AK,Synolon Blue AK,Dorosperse B/K KKL,Dorosperse Dark Grey KKL,Dorosperse Brown K-3LR,Dianix Black AM-SLR,Dianix Grey AM-SLR,Dianix Yellow Brown AM-R,Synolon Black AK-NB,Synolon Dark Grey AK,Synofast 28
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,0.267185,0.267185,0.267185,0.184175,0.184175,0.184175,0.092698,0.092698,0.092698,0.149968,0.149968,0.150284,0.149968,0.149968,0.150284,0.058491,0.058491,0.058807
std,0.959340,0.959340,0.959340,0.861210,0.861210,0.861210,0.558839,0.558839,0.558839,0.828595,0.828595,0.834707,0.828595,0.828595,0.834707,0.500926,0.500926,0.511031
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000


In [89]:
np.array(data.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_형상',
       '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도', '위사_형상',
       '제직_원단구조', '제직_조직', '제직_중량_생지', '제직_폭_생지', 'Pilot 전처리 기계장비명',
       'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명', 'Pilot 전처리 조제 #1제조사명',
       'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량', 'Pilot 전처리 조제 #2 제품명',
       'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류', 'Pilot 전처리 조제 #2투입량',
       'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명', 'Pilot 전처리 조제 #3 종류',
       'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도', 'Pilot 전처리 상승속도 #1',
       'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간', 'Pilot 전처리 상승속도 #2',
       'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간', 'Pilot 전처리 상승속도 #3',
       'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간', 'Pilot 전처리 하강속도 #1',
       'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간', 'Pilot 전처리 하강속도 #2',
       'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간', 'Pilot 전처리 하강속도 #3',
       'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 

In [90]:
배합2=data[[ 'Lab 배합 조제 #1 명', 'Lab 배합 조제 #1 투입량',
             'Lab 배합 조제 #2 명', 'Lab 배합 조제 #2 투입량',
             'Lab 배합 조제 #3 명', 'Lab 배합 조제 #3 투입량',
             'Lab 배합 조제 #4 명', 'Lab 배합 조제 #4 투입량']]
배합2

,Lab 배합 조제 #1 명,Lab 배합 조제 #1 투입량,Lab 배합 조제 #2 명,Lab 배합 조제 #2 투입량,Lab 배합 조제 #3 명,Lab 배합 조제 #3 투입량,Lab 배합 조제 #4 명,Lab 배합 조제 #4 투입량
0,Sunsolt RM-340S,0.3,빙초산,0.2,NaN,NaN,NaN,NaN
1,Sunsolt RM-340S,0.3,빙초산,0.2,NaN,NaN,NaN,NaN
2,Sunsolt RM-340S,0.3,빙초산,0.2,NaN,NaN,NaN,NaN
3,Sunsolt RM-340S,0.3,빙초산,0.2,NaN,NaN,NaN,NaN
4,Sunsolt RM-340S,0.3,빙초산,0.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3163,Sunsolt RM-340S,0.3,빙초산,0.2,UVK-400 H/C,3,NICCA FI-NONE HF-0339,6
3164,Sunsolt RM-340S,0.3,빙초산,0.2,UVK-400 H/C,3,NICCA FI-NONE HF-0339,6
3165,Sunsolt RM-340S,0.3,빙초산,0.2,UVK-400 H/C,3,NICCA FI-NONE HF-0339,6
3166,Sunsolt RM-340S,0.3,빙초산,0.2,UVK-400 H/C,3,NICCA FI-NONE HF-0339,6


In [91]:
배합2_df=pivot_PET(배합2)
배합2_df=배합2_df.fillna(0)
배합2_df.columns='배합_'+배합2_df.columns
배합2_df=배합2_df.drop(['배합_NICCA FI-NONE HF-0339'],axis=1)
배합2_df

,배합_Sunsolt RM-340S,배합_빙초산,배합_Fadex® TS liq,배합_Sera Fast P-UVC,배합_UVK-400 H/C
0,0.3,0.2,0,0,0
1,0.3,0.2,0,0,0
2,0.3,0.2,0,0,0
3,0.3,0.2,0,0,0
4,0.3,0.2,0,0,0
...,...,...,...,...,...
3163,0.3,0.2,0,0,3
3164,0.3,0.2,0,0,3
3165,0.3,0.2,0,0,3
3166,0.3,0.2,0,0,3


In [92]:
배합2_df.describe()

,배합_Sunsolt RM-340S,배합_빙초산,배합_Fadex® TS liq,배합_Sera Fast P-UVC,배합_UVK-400 H/C
count,3.168000e+03,3168.0,3168.000000,3168.000000,3168.000000
mean,3.000000e-01,0.2,0.166667,0.166667,0.166667
std,5.551991e-17,0.0,0.687293,0.687293,0.687293
min,3.000000e-01,0.2,0.000000,0.000000,0.000000
25%,3.000000e-01,0.2,0.000000,0.000000,0.000000
50%,3.000000e-01,0.2,0.000000,0.000000,0.000000
75%,3.000000e-01,0.2,0.000000,0.000000,0.000000
max,3.000000e-01,0.2,4.000000,4.000000,4.000000


In [93]:
배합=pd.concat([배합_df,배합2_df],axis=1)
배합

,Dorosperse Red KKL,Dorosperse Yellow KKL,Dorosperse Blue KKL,Dianix Red AM-SLR,Dianix Yellow AM-2R,Dianix Blue AM-2G,Synolon Red AK,Synolon Yellow AK,Synolon Blue AK,Dorosperse B/K KKL,...,Dianix Grey AM-SLR,Dianix Yellow Brown AM-R,Synolon Black AK-NB,Synolon Dark Grey AK,Synofast 28,배합_Sunsolt RM-340S,배합_빙초산,배합_Fadex® TS liq,배합_Sera Fast P-UVC,배합_UVK-400 H/C
0,0.017,0.000,0.033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,0
1,0.017,0.000,0.033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,0
2,0.033,0.000,0.017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,0
3,0.033,0.000,0.017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,0
4,0.000,0.017,0.033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,3
3164,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,3
3165,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,3
3166,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.3,0.2,0,0,3


# 4. 염색

In [94]:
염색=data[[#'Lab 염색 기계명', 
         'Lab 염색 시작온도',
       'Lab 염색 상승속도 #1', 'Lab 염색 상승온도 #1', 'Lab 염색 상승온도 #1 유지시간',
       'Lab 염색 상승속도 #2', 'Lab 염색 상승온도 #2', 'Lab 염색 상승온도 #2 유지시간',
       'Lab 염색 상승속도 #3', 'Lab 염색 상승온도 #3', 'Lab 염색 상승온도 #3 유지시간',
       'Lab 염색 하강속도 #1', 'Lab 염색 하강온도 #1', 'Lab 염색 하강온도 #1 유지시간',
       'Lab 염색 하강속도 #2', 'Lab 염색 하강온도 #2', 'Lab 염색 하강온도 #2 유지시간',
       'Lab 염색 하강속도 #3', 'Lab 염색 하강온도 #3', 'Lab 염색 하강온도 #3 유지시간',
       'Lab 염색 종료속도', 'Lab 염색 종료온도', 'Lab 염색 종료온도 유지시간']]

In [95]:
염색

,Lab 염색 시작온도,Lab 염색 상승속도 #1,Lab 염색 상승온도 #1,Lab 염색 상승온도 #1 유지시간,Lab 염색 상승속도 #2,Lab 염색 상승온도 #2,Lab 염색 상승온도 #2 유지시간,Lab 염색 상승속도 #3,Lab 염색 상승온도 #3,Lab 염색 상승온도 #3 유지시간,...,Lab 염색 하강온도 #1 유지시간,Lab 염색 하강속도 #2,Lab 염색 하강온도 #2,Lab 염색 하강온도 #2 유지시간,Lab 염색 하강속도 #3,Lab 염색 하강온도 #3,Lab 염색 하강온도 #3 유지시간,Lab 염색 종료속도,Lab 염색 종료온도,Lab 염색 종료온도 유지시간
0,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
1,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
2,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
3,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
4,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
3164,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
3165,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0
3166,40,1.5,64,0,0.75,100,0,1.458333,135,60,...,0,0,64,0,0,64,0,0,64,0


In [96]:
np.array(data.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_형상',
       '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도', '위사_형상',
       '제직_원단구조', '제직_조직', '제직_중량_생지', '제직_폭_생지', 'Pilot 전처리 기계장비명',
       'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명', 'Pilot 전처리 조제 #1제조사명',
       'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량', 'Pilot 전처리 조제 #2 제품명',
       'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류', 'Pilot 전처리 조제 #2투입량',
       'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명', 'Pilot 전처리 조제 #3 종류',
       'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도', 'Pilot 전처리 상승속도 #1',
       'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간', 'Pilot 전처리 상승속도 #2',
       'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간', 'Pilot 전처리 상승속도 #3',
       'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간', 'Pilot 전처리 하강속도 #1',
       'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간', 'Pilot 전처리 하강속도 #2',
       'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간', 'Pilot 전처리 하강속도 #3',
       'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 

# 5. 후처리

In [97]:
#후처리 조제 전처리
후처리=data[['Lab 후처리 조제 #1 명', 'Lab 후처리 조제 #1 투입량', 'Lab 후처리 조제 #2 명', 'Lab 후처리 조제 #2 투입량']]
후처리

,Lab 후처리 조제 #1 명,Lab 후처리 조제 #1 투입량,Lab 후처리 조제 #2 명,Lab 후처리 조제 #2 투입량
0,NaOH,2,Sera Con M-FAS,4
1,NaOH,2,Sera Con M-FAS,4
2,NaOH,2,Sera Con M-FAS,4
3,NaOH,2,Sera Con M-FAS,4
4,NaOH,2,Sera Con M-FAS,4
...,...,...,...,...
3163,NaOH,2,Sera Con M-FAS,4
3164,NaOH,2,Sera Con M-FAS,4
3165,NaOH,2,Sera Con M-FAS,4
3166,NaOH,2,Sera Con M-FAS,4


In [98]:
후처리_df=pivot_PET(후처리)
후처리_df.columns='후처리_'+후처리_df.columns
후처리_df

,후처리_NaOH,후처리_Sera Con M-FAS
0,2,4
1,2,4
2,2,4
3,2,4
4,2,4
...,...,...
3163,2,4
3164,2,4
3165,2,4
3166,2,4


In [99]:
# 3번째 수세변수 3개('Lab 후처리 수세온도 #3','Lab 후처리 수세시간 #3', 'Lab 후처리 수세횟수 #3') 제거
# 수세 시간변수 3개('Lab 후처리 수세시간 #1','Lab 후처리 수세시간 #2','Lab 후처리 수세시간 #3') 제거
후처리_df2=data[[#'Lab 후처리 기계명','Lab 후처리 환원세정 용수량', 
       'Lab 후처리 환원세정 시작온도', 'Lab 후처리 환원세정 상승속도 #1',
       'Lab 후처리 환원세정 상승온도 #1', 'Lab 후처리 환원세정 상승온도 #1 유지시간',
       'Lab 후처리 환원세정 상승속도 #2', 'Lab 후처리 환원세정 상승온도 #2',
       'Lab 후처리 환원세정 상승온도 #2 유지시간', 'Lab 후처리 환원세정 상승속도 #3',
       'Lab 후처리 환원세정 상승온도 #3', 'Lab 후처리 환원세정 상승온도 #3 유지시간',
       'Lab 후처리 환원세정 하강속도 #1', 'Lab 후처리 환원세정 하강온도 #1',
       'Lab 후처리 환원세정 하강온도 #1 유지시간', 'Lab 후처리 환원세정 하강속도 #2',
       'Lab 후처리 환원세정 하강온도 #2', 'Lab 후처리 환원세정 하강온도 #2 유지시간',
       'Lab 후처리 환원세정 하강속도 #3', 'Lab 후처리 환원세정 하강온도 #3',
       'Lab 후처리 환원세정 하강온도 #3 유지시간', 'Lab 후처리 환원세정 종료속도',
       'Lab 후처리 환원세정 종료온도', 'Lab 후처리 환원세정 종료온도 유지시간', 'Lab 후처리 수세온도 #1',
       'Lab 후처리 수세횟수 #1', 'Lab 후처리 수세온도 #2', 'Lab 후처리 수세횟수 #2', 'Lab 후처리 건조온도','Lab 후처리 건조시간']]
후처리_df2

,Lab 후처리 환원세정 시작온도,Lab 후처리 환원세정 상승속도 #1,Lab 후처리 환원세정 상승온도 #1,Lab 후처리 환원세정 상승온도 #1 유지시간,Lab 후처리 환원세정 상승속도 #2,Lab 후처리 환원세정 상승온도 #2,Lab 후처리 환원세정 상승온도 #2 유지시간,Lab 후처리 환원세정 상승속도 #3,Lab 후처리 환원세정 상승온도 #3,Lab 후처리 환원세정 상승온도 #3 유지시간,...,Lab 후처리 환원세정 하강온도 #3 유지시간,Lab 후처리 환원세정 종료속도,Lab 후처리 환원세정 종료온도,Lab 후처리 환원세정 종료온도 유지시간,Lab 후처리 수세온도 #1,Lab 후처리 수세횟수 #1,Lab 후처리 수세온도 #2,Lab 후처리 수세횟수 #2,Lab 후처리 건조온도,Lab 후처리 건조시간
0,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
1,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
2,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
3,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
4,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
3164,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
3165,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10
3166,40,2,80,20,0,80,0,0,80,0,...,0,0,80,0,60,2,25,1,100,10


In [100]:
후처리=pd.concat([후처리_df,후처리_df2],axis=1)
후처리

,후처리_NaOH,후처리_Sera Con M-FAS,Lab 후처리 환원세정 시작온도,Lab 후처리 환원세정 상승속도 #1,Lab 후처리 환원세정 상승온도 #1,Lab 후처리 환원세정 상승온도 #1 유지시간,Lab 후처리 환원세정 상승속도 #2,Lab 후처리 환원세정 상승온도 #2,Lab 후처리 환원세정 상승온도 #2 유지시간,Lab 후처리 환원세정 상승속도 #3,...,Lab 후처리 환원세정 하강온도 #3 유지시간,Lab 후처리 환원세정 종료속도,Lab 후처리 환원세정 종료온도,Lab 후처리 환원세정 종료온도 유지시간,Lab 후처리 수세온도 #1,Lab 후처리 수세횟수 #1,Lab 후처리 수세온도 #2,Lab 후처리 수세횟수 #2,Lab 후처리 건조온도,Lab 후처리 건조시간
0,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
1,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
2,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
3,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
4,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
3164,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
3165,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10
3166,2,4,40,2,80,20,0,80,0,0,...,0,0,80,0,60,2,25,1,100,10


# 6. 후가공

In [101]:
#현재 후가공시, 조제는 넣지 않으므로 우선, 변수 삭제
후가공=data[[#'Lab 후가공 패더장비명',
       'Lab 후가공 패더압력', 'Lab 후가공 패더속도', 
    #'Lab 후가공 텐터장비명', 
    'Lab 후가공 건조온도',
       'Lab 후가공 건조시간', 'Lab 후가공 열풍', 'Lab 후가공 스팀']]
후가공

,Lab 후가공 패더압력,Lab 후가공 패더속도,Lab 후가공 건조온도,Lab 후가공 건조시간,Lab 후가공 열풍,Lab 후가공 스팀
0,1,1,180,1,2,2
1,1,1,180,1,2,2
2,1,1,180,1,2,2
3,1,1,180,1,2,2
4,1,1,180,1,2,2
...,...,...,...,...,...,...
3163,1,1,180,1,2,2
3164,1,1,180,1,2,2
3165,1,1,180,1,2,2
3166,1,1,180,1,2,2


# 7. 검사

In [102]:
np.array(data.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_형상',
       '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성', '위사_밀도', '위사_형상',
       '제직_원단구조', '제직_조직', '제직_중량_생지', '제직_폭_생지', 'Pilot 전처리 기계장비명',
       'Pilot 전처리 원단 투입량', 'Pilot 전처리 조제 #1 제품명', 'Pilot 전처리 조제 #1제조사명',
       'Pilot 전처리 조제 #1 종류', 'Pilot 전처리 조제 #1투입량', 'Pilot 전처리 조제 #2 제품명',
       'Pilot 전처리 조제 #2제조사명', 'Pilot 전처리 조제 #2 종류', 'Pilot 전처리 조제 #2투입량',
       'Pilot 전처리 조제 #3 제품명', 'Pilot 전처리 조제 #3제조사명', 'Pilot 전처리 조제 #3 종류',
       'Pilot 전처리 조제 #3투입량', 'Pilot 전처리 시작온도', 'Pilot 전처리 상승속도 #1',
       'Pilot 전처리 상승온도 #1', 'Pilot 전처리 상승온도 #1 유지시간', 'Pilot 전처리 상승속도 #2',
       'Pilot 전처리 상승온도 #2', 'Pilot 전처리 상승온도 #2 유지시간', 'Pilot 전처리 상승속도 #3',
       'Pilot 전처리 상승온도 #3', 'Pilot 전처리 상승온도 #3 유지시간', 'Pilot 전처리 하강속도 #1',
       'Pilot 전처리 하강온도 #1', 'Pilot 전처리 하강온도 #1 유지시간', 'Pilot 전처리 하강속도 #2',
       'Pilot 전처리 하강온도 #2', 'Pilot 전처리 하강온도 #2 유지시간', 'Pilot 전처리 하강속도 #3',
       'Pilot 전처리 하강온도 #3', 'Pilot 전처리 하강온도 

In [103]:
검사=data[['잔욕염색 검사_L*','잔욕염색 검사_a*', '잔욕염색 검사_b*', '잔욕염색 검사_K/S']]#, 'Lab 후가공 후 검사_K/S','내광성 시험 후 검사_내광성등급']]
검사

,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,93.79,-0.1,2.55,0.0415
1,93.96,-0.12,2.5,0.0396
2,94.12,-0.02,2.87,0.0412
3,93.84,0.16,2.85,0.0435
4,94.08,-0.19,2.53,0.0398
...,...,...,...,...
3163,82.9728,-3.5983,1.5226,0.21
3164,77.4101,-6.2102,-3.6497,0.31
3165,77.5606,-6.191,-4.2008,0.31
3166,72.4137,-7.2742,-7.7309,0.54


In [104]:
data_전처리=pd.concat([소재,전처리,배합,염색,후처리,후가공,검사],axis=1)
data_전처리

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,Lab 후가공 패더압력,Lab 후가공 패더속도,Lab 후가공 건조온도,Lab 후가공 건조시간,Lab 후가공 열풍,Lab 후가공 스팀,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,93.79,-0.1,2.55,0.0415
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,93.96,-0.12,2.5,0.0396
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,94.12,-0.02,2.87,0.0412
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,93.84,0.16,2.85,0.0435
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,94.08,-0.19,2.53,0.0398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,82.9728,-3.5983,1.5226,0.21
3164,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,77.4101,-6.2102,-3.6497,0.31
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,77.5606,-6.191,-4.2008,0.31
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,180,1,2,2,72.4137,-7.2742,-7.7309,0.54


In [105]:
#데이터중 na값이 있는지 검사
np.array(np.sum(data_전처리.isnull()))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int64)

# 염료 구성비 파생변수 추가

In [4]:
data_전처리=data_전처리[data_전처리[['Dorosperse Yellow KKL', 'Dorosperse Red KKL',
       'Dorosperse Blue KKL', 'Dianix Yellow AM-2R', 'Dianix Red AM-SLR',
       'Dianix Blue AM-2G', 'Synolon Yellow AK', 'Synolon Red AK',
       'Synolon Blue AK', 'Dorosperse B/K KKL', 'Dianix Black AM-SLR',
       'Synolon Black AK-NB', 'Dorosperse Dark Grey KKL',
       'Dianix Grey AM-SLR', 'Synolon Dark Grey AK',
       'Dianix Yellow Brown AM-R', 'Dorosperse Brown K-3LR',
       'Synofast 28']].sum(axis=1)!=0]

NameError: name 'data_전처리' is not defined

In [5]:
data

NameError: name 'data' is not defined

In [6]:
color_mat = data_전처리[['Dorosperse Yellow KKL', 'Dorosperse Red KKL',
       'Dorosperse Blue KKL', 'Dianix Yellow AM-2R', 'Dianix Red AM-SLR',
       'Dianix Blue AM-2G', 'Synolon Yellow AK', 'Synolon Red AK',
       'Synolon Blue AK', 'Dorosperse B/K KKL', 'Dianix Black AM-SLR',
       'Synolon Black AK-NB', 'Dorosperse Dark Grey KKL',
       'Dianix Grey AM-SLR', 'Synolon Dark Grey AK',
       'Dianix Yellow Brown AM-R', 'Dorosperse Brown K-3LR',
       'Synofast 28']].values

NameError: name 'data_전처리' is not defined

In [7]:
color_mat

NameError: name 'color_mat' is not defined

In [110]:
col = ['Dorosperse Yellow KKL', 'Dorosperse Red KKL',
       'Dorosperse Blue KKL', 'Dianix Yellow AM-2R', 'Dianix Red AM-SLR',
       'Dianix Blue AM-2G', 'Synolon Yellow AK', 'Synolon Red AK',
       'Synolon Blue AK', 'Dorosperse B/K KKL', 'Dianix Black AM-SLR',
       'Synolon Black AK-NB', 'Dorosperse Dark Grey KKL',
       'Dianix Grey AM-SLR', 'Synolon Dark Grey AK',
       'Dianix Yellow Brown AM-R', 'Dorosperse Brown K-3LR',
       'Synofast 28']

In [111]:
data_전처리[[i+"_prop" for i in col]] =color_mat/color_mat.sum(axis=1).reshape(-1,1)

In [112]:
prop=data_전처리[[ 'Dorosperse Yellow KKL_prop',
       'Dorosperse Red KKL_prop', 'Dorosperse Blue KKL_prop',
       'Dianix Yellow AM-2R_prop', 'Dianix Red AM-SLR_prop',
       'Dianix Blue AM-2G_prop', 'Synolon Yellow AK_prop',
       'Synolon Red AK_prop', 'Synolon Blue AK_prop',
       'Dorosperse B/K KKL_prop', 'Dianix Black AM-SLR_prop',
       'Synolon Black AK-NB_prop', 'Dorosperse Dark Grey KKL_prop',
       'Dianix Grey AM-SLR_prop', 'Synolon Dark Grey AK_prop',
       'Dianix Yellow Brown AM-R_prop', 'Dorosperse Brown K-3LR_prop',
       'Synofast 28_prop']]
prop.describe()


,Dorosperse Yellow KKL_prop,Dorosperse Red KKL_prop,Dorosperse Blue KKL_prop,Dianix Yellow AM-2R_prop,Dianix Red AM-SLR_prop,Dianix Blue AM-2G_prop,Synolon Yellow AK_prop,Synolon Red AK_prop,Synolon Blue AK_prop,Dorosperse B/K KKL_prop,Dianix Black AM-SLR_prop,Synolon Black AK-NB_prop,Dorosperse Dark Grey KKL_prop,Dianix Grey AM-SLR_prop,Synolon Dark Grey AK_prop,Dianix Yellow Brown AM-R_prop,Dorosperse Brown K-3LR_prop,Synofast 28_prop
count,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000,3168.000000
mean,0.101221,0.101221,0.101221,0.070286,0.070286,0.070286,0.036195,0.036195,0.036195,0.053662,0.053662,0.019571,0.053662,0.053662,0.019571,0.052399,0.052399,0.018308
std,0.263076,0.263076,0.263076,0.241551,0.241551,0.241551,0.166963,0.166963,0.166963,0.225384,0.225384,0.138542,0.225384,0.225384,0.138542,0.222865,0.222865,0.134084
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [113]:
#종속변수를 맨 오른쪽으로 위치 변경
검사=data_전처리[['잔욕염색 검사_L*','잔욕염색 검사_a*', '잔욕염색 검사_b*', '잔욕염색 검사_K/S']]
data_전처리=data_전처리.drop(['잔욕염색 검사_L*','잔욕염색 검사_a*', '잔욕염색 검사_b*', '잔욕염색 검사_K/S'],axis=1)
data_전처리=pd.concat([data_전처리,검사],axis=1)
data_전처리

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,Dorosperse Dark Grey KKL_prop,Dianix Grey AM-SLR_prop,Synolon Dark Grey AK_prop,Dianix Yellow Brown AM-R_prop,Dorosperse Brown K-3LR_prop,Synofast 28_prop,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,93.79,-0.1,2.55,0.0415
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,93.96,-0.12,2.5,0.0396
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,94.12,-0.02,2.87,0.0412
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,93.84,0.16,2.85,0.0435
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,94.08,-0.19,2.53,0.0398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,82.9728,-3.5983,1.5226,0.21
3164,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,77.4101,-6.2102,-3.6497,0.31
3165,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,77.5606,-6.191,-4.2008,0.31
3166,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,72.4137,-7.2742,-7.7309,0.54


# 중복데이터 평균내기

In [114]:
ch=["a"]*len(data_전처리.columns)

In [115]:
nm=[]
for i in range(len(ch)):
    nm.append(ch[i]+str(i))
gan_columns=nm

In [116]:
origin_columns=data_전처리.columns

In [117]:
data_전처리.columns=gan_columns

In [118]:
data_pret=data_전처리

In [119]:
np.array(origin_columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_형상',
       '경사_성상_섬도 #1', '경사_성상_섬도 #2', '경사_성상_섬도 #3', '경사_성상_섬도 #4',
       '경사_성상_비율 #1', '경사_성상_비율 #2', '경사_성상_비율 #3', '경사_성상_비율 #4',
       '경사_성상_특성', '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성',
       '위사_밀도', '위사_형상', '위사_성상_섬도 #1', '위사_성상_섬도 #2', '위사_성상_섬도 #3',
       '위사_성상_섬도 #4', '위사_성상_비율 #1', '위사_성상_비율 #2', '위사_성상_비율 #3',
       '위사_성상_비율 #4', '위사_성상_특성', '제직_원단구조', '제직_조직', '제직_중량_생지',
       '제직_폭_생지', '전처리_Sera Wash C-NEP', '전처리_NaOH', '전처리_CH3COOH',
       'Pilot 전처리 시작온도', 'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1',
       'Pilot 전처리 상승온도 #1 유지시간', 'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2',
       'Pilot 전처리 상승온도 #2 유지시간', 'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3',
       'Pilot 전처리 상승온도 #3 유지시간', 'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1',
       'Pilot 전처리 하강온도 #1 유지시간', 'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2',
       'Pilot 전처리 하강온도 #2 유지시간', 'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3',
       'Pil

In [120]:
np.array(data_전처리.columns)

array(['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10',
       'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19',
       'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28',
       'a29', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37',
       'a38', 'a39', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46',
       'a47', 'a48', 'a49', 'a50', 'a51', 'a52', 'a53', 'a54', 'a55',
       'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63', 'a64',
       'a65', 'a66', 'a67', 'a68', 'a69', 'a70', 'a71', 'a72', 'a73',
       'a74', 'a75', 'a76', 'a77', 'a78', 'a79', 'a80', 'a81', 'a82',
       'a83', 'a84', 'a85', 'a86', 'a87', 'a88', 'a89', 'a90', 'a91',
       'a92', 'a93', 'a94', 'a95', 'a96', 'a97', 'a98', 'a99', 'a100',
       'a101', 'a102', 'a103', 'a104', 'a105', 'a106', 'a107', 'a108',
       'a109', 'a110', 'a111', 'a112', 'a113', 'a114', 'a115', 'a116',
       'a117', 'a118', 'a119', 'a120', 'a121', 'a122', 'a123', 'a124',
       'a125

In [122]:
q='''
select a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10,a11, a12, a13, a14, a15, a16, a17, a18, a19,a20, a21, a22, a23, a24, a25, a26, a27, a28,a29, a30, a31, a32, a33, a34, a35, a36, a37,a38, a39, a40, a41, a42, a43, a44, a45, a46,a47, a48, a49, a50, a51, a52, a53, a54, a55,a56, a57, a58, a59, a60, a61, a62, a63, a64,a65, a66, a67, a68, a69, a70, a71, a72, a73,a74, a75, a76, a77, a78, a79, a80, a81, a82,a83, a84, a85, a86, a87, a88, a89, a90, a91,a92, a93, a94, a95, a96, a97, a98, a99, a100,a101, a102, a103, a104, a105, a106, a107, a108,a109, a110, a111, a112, a113, a114, a115, a116,a117, a118, a119, a120, a121, a122, a123, a124,a125, a126, a127, a128, a129, a130, a131, a132,a133, a134, a135, a136, a137, a138, a139, a140,a141, a142, a143, a144, a145, a146, a147, a148,a149, a150, a151, a152, a153, a154, a155, a156,a157, a158, a159, a160, a161, a162, a163, a164,a165, a166, a167, a168, avg(a169), avg(a170), avg(a171), avg(a172)
from data_pret
group by a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10,a11, a12, a13, a14, a15, a16, a17, a18, a19,a20, a21, a22, a23, a24, a25, a26, a27, a28,a29, a30, a31, a32, a33, a34, a35, a36, a37,a38, a39, a40, a41, a42, a43, a44, a45, a46,a47, a48, a49, a50, a51, a52, a53, a54, a55,a56, a57, a58, a59, a60, a61, a62, a63, a64,a65, a66, a67, a68, a69, a70, a71, a72, a73,a74, a75, a76, a77, a78, a79, a80, a81, a82,a83, a84, a85, a86, a87, a88, a89, a90, a91,a92, a93, a94, a95, a96, a97, a98, a99, a100,a101, a102, a103, a104, a105, a106, a107, a108,a109, a110, a111, a112, a113, a114, a115, a116,a117, a118, a119, a120, a121, a122, a123, a124,a125, a126, a127, a128, a129, a130, a131, a132,a133, a134, a135, a136, a137, a138, a139, a140,a141, a142, a143, a144, a145, a146, a147, a148,a149, a150, a151, a152, a153, a154, a155, a156,a157, a158, a159, a160, a161, a162, a163, a164,a165, a166, a167, a168
'''
q=sqldf(q)
#print(sqldf(q, locals()))

In [123]:
q.columns=origin_columns
q

,경사_소재명,경사_섬도,경사_필라멘트수,경사_광택,경사_특성,경사_밀도,경사_형상,경사_성상_섬도 #1,경사_성상_섬도 #2,경사_성상_섬도 #3,...,Dorosperse Dark Grey KKL_prop,Dianix Grey AM-SLR_prop,Synolon Dark Grey AK_prop,Dianix Yellow Brown AM-R_prop,Dorosperse Brown K-3LR_prop,Synofast 28_prop,잔욕염색 검사_L*,잔욕염색 검사_a*,잔욕염색 검사_b*,잔욕염색 검사_K/S
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,1.0,82.260000,1.390000,4.150000,0.0655
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,1.0,82.255000,1.400000,4.135000,0.0835
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,1.0,54.690000,9.225000,9.180000,2.3030
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,1.0,55.525000,9.180000,9.320000,2.1910
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,1.0,54.715000,9.210000,9.160000,2.2970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,76.980000,24.935000,6.915000,0.5070
1436,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,75.690025,27.029775,4.270375,0.5775
1437,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,77.176775,25.206600,3.949725,0.4875
1438,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,76.542050,25.546800,4.672800,0.5300


In [124]:
np.array(q.columns)

array(['경사_소재명', '경사_섬도', '경사_필라멘트수', '경사_광택', '경사_특성', '경사_밀도', '경사_형상',
       '경사_성상_섬도 #1', '경사_성상_섬도 #2', '경사_성상_섬도 #3', '경사_성상_섬도 #4',
       '경사_성상_비율 #1', '경사_성상_비율 #2', '경사_성상_비율 #3', '경사_성상_비율 #4',
       '경사_성상_특성', '위사_소재명', '위사_섬도', '위사_필라멘트수', '위사_광택', '위사_특성',
       '위사_밀도', '위사_형상', '위사_성상_섬도 #1', '위사_성상_섬도 #2', '위사_성상_섬도 #3',
       '위사_성상_섬도 #4', '위사_성상_비율 #1', '위사_성상_비율 #2', '위사_성상_비율 #3',
       '위사_성상_비율 #4', '위사_성상_특성', '제직_원단구조', '제직_조직', '제직_중량_생지',
       '제직_폭_생지', '전처리_Sera Wash C-NEP', '전처리_NaOH', '전처리_CH3COOH',
       'Pilot 전처리 시작온도', 'Pilot 전처리 상승속도 #1', 'Pilot 전처리 상승온도 #1',
       'Pilot 전처리 상승온도 #1 유지시간', 'Pilot 전처리 상승속도 #2', 'Pilot 전처리 상승온도 #2',
       'Pilot 전처리 상승온도 #2 유지시간', 'Pilot 전처리 상승속도 #3', 'Pilot 전처리 상승온도 #3',
       'Pilot 전처리 상승온도 #3 유지시간', 'Pilot 전처리 하강속도 #1', 'Pilot 전처리 하강온도 #1',
       'Pilot 전처리 하강온도 #1 유지시간', 'Pilot 전처리 하강속도 #2', 'Pilot 전처리 하강온도 #2',
       'Pilot 전처리 하강온도 #2 유지시간', 'Pilot 전처리 하강속도 #3', 'Pilot 전처리 하강온도 #3',
       'Pil

# 데이터 내보내기

In [125]:
q.to_csv(direc+'UI개발데이터.csv',encoding='euc-kr')